In [1]:
# Лабораторная работа 1 по дисциплине МРЗвИС
# Выполнена студентом группы 121702
# БГУИР Заломов Роман Андреевич
#
# Вариант 15: Реализовать модель линейной рециркуляционной сети 
# с постоянным коэффициентом обучения и нормированными весовыми коэффициентами.
#
# Данный файл содержит алгоритмы построения графиков различных зависимостей

In [2]:
from lrnn import (
    LRNN,
    image_to_blocks,
    compress_image,
    decompress_image
)
import numpy as np
from PIL import Image
import plotly.graph_objects as go

In [3]:
MAX_RGB_VALUE = 255
COLOR_CHANNELS_AMOUNT = 3

In [4]:
# Collecting everything

block_width = 10
block_height = 10

n = block_height * block_width
# Hidden layer neuron amount
p = 20 

img = Image.open('car.jpg')
img_array = np.asarray(img)
shape = img_array.shape
blocks = image_to_blocks(img_array, block_height, block_width, overlap=0)

l = len(blocks)
# Compression coeff
print('Z =', (n*l) / ((n+l) * p+2))

color_df = ((2 * blocks / MAX_RGB_VALUE) - 1).reshape(len(blocks), -1, 3).transpose(0, 2, 1).reshape(-1, block_height * block_width)
train = np.matrix(color_df[np.random.choice(color_df.shape[0], int(color_df.shape[0] * 0.1))])


network = LRNN(n, p, 0.0015)
network.train(train, 15000, learn_by_loss=True, max_loss=2000)

compressed = compress_image(network.W_enc, img_array, COLOR_CHANNELS_AMOUNT, block_height, block_width, 0)

compression_info_size = (
    compressed.size * compressed.itemsize +
    network.W_dec.size * network.W_dec.itemsize +
    np.array(shape).size * np.array(shape).itemsize +
    np.array((block_height, block_width)).size * np.array((block_height, block_width)).itemsize 
) * 8
print(f'Compression coefficient: {(img_array.size * img_array.itemsize * 8) / compression_info_size}')
print('Z =', (n*l) / ((n+l) * p + 2))

dimg = decompress_image(network.W_dec, compressed, shape, COLOR_CHANNELS_AMOUNT, block_height, block_width, 0)
dimg_array = np.asarray(dimg)
dimg.save('compression-decompression_test.jpg')

Z = 4.807507403561401
Epoch 1/15000, Loss: 2205.8565997210676
Epoch 2/15000, Loss: 2083.57631424727
Epoch 3/15000, Loss: 1988.3489490527409
Compression coefficient: 1.2334714821393329
Z = 4.807507403561401


## Relation between learning rate and epochs 

In [27]:
alphas = np.linspace(0.0001, 0.004, endpoint=True, num=10)
epochs_amount_mean: list[int] = []
fig = go.Figure()
for alpha in alphas:
    print(alpha)
    dl = LRNN(n, p, alpha)
    dl.train(train, 15000, learn_by_loss=True, max_loss=1200)
    epochs_amount_mean.append(dl.epoch)
fig.add_trace(go.Scatter(x=alphas, y=epochs_amount_mean, mode='lines+markers',
                         name='Среднее значение'))
fig.update_layout(title='Зависимость количества итераций обучения от коэффициента обучения',
                  xaxis_title='Коэффициент обучения',
                  yaxis_title='Количество итераций обучения',
                  height=720, width=1280,
                  font=dict(size=20))
fig.show()

0.0001
Epoch 1/15000, Loss: 4623.271515342316
Epoch 2/15000, Loss: 3387.6313422687695
Epoch 3/15000, Loss: 3119.344802446319
Epoch 4/15000, Loss: 2977.0820787764987
Epoch 5/15000, Loss: 2874.8121385762947
Epoch 6/15000, Loss: 2793.9637341718144
Epoch 7/15000, Loss: 2727.244814478598
Epoch 8/15000, Loss: 2670.7761893285688
Epoch 9/15000, Loss: 2622.132377550569
Epoch 10/15000, Loss: 2579.6480835428642
Epoch 11/15000, Loss: 2542.10941157143
Epoch 12/15000, Loss: 2508.6005351073327
Epoch 13/15000, Loss: 2478.41093010318
Epoch 14/15000, Loss: 2450.9813244434913
Epoch 15/15000, Loss: 2425.8655584319154
Epoch 16/15000, Loss: 2402.7034816706505
Epoch 17/15000, Loss: 2381.202653651792
Epoch 18/15000, Loss: 2361.1229064766976
Epoch 19/15000, Loss: 2342.2670275557025
Epoch 20/15000, Loss: 2324.4697899184407
Epoch 21/15000, Loss: 2307.5932010990437
Epoch 22/15000, Loss: 2291.5210901898577
Epoch 23/15000, Loss: 2276.1554567053586
Epoch 24/15000, Loss: 2261.411972738323
Epoch 25/15000, Loss: 2247.2

## Relationship between hidden layer dimension and epochs

In [15]:
p_s = (6, 8, 10, 12, 14, 16, 18, 20)
epochs_amount: list[int] = []
compression_rates: list[float] = []
for p in p_s:    
    dl = LRNN(n, p, 0.001)
    dl.train(train, 15000, learn_by_loss=True, max_loss=1900)
    epochs_amount.append(dl.epoch)

    compressed = compress_image(dl.W_enc, img_array, COLOR_CHANNELS_AMOUNT, block_height, block_width, 0)
    compression_info_size = (
        compressed.size * compressed.itemsize +
        network.W_dec.size * network.W_dec.itemsize +
        np.array(shape).size * np.array(shape).itemsize +
        np.array((block_height, block_width)).size * np.array((block_height, block_width)).itemsize 
    ) * 8
    compression_rates.append((img_array.size * img_array.itemsize * 8) / compression_info_size)

fig = go.Figure()
fig.add_trace(go.Scatter(x=compression_rates, y=epochs_amount, mode='lines+markers'))
fig.update_layout(title='Зависимость количества итераций обучения от коэффициента сжатия',
                  xaxis_title='Коэффициент сжатия',
                  yaxis_title='Количество итераций обучения',
                  height=720, width=1280,
                  font=dict(size=20))
fig.show()

Epoch 1/15000, Loss: 3409.6066457136185
Epoch 2/15000, Loss: 2570.02669691301
Epoch 3/15000, Loss: 2328.712127521082
Epoch 4/15000, Loss: 2220.0310075069992
Epoch 5/15000, Loss: 2159.1259253490757
Epoch 6/15000, Loss: 2120.613921003126
Epoch 7/15000, Loss: 2094.3845563013206
Epoch 8/15000, Loss: 2075.5934013450487
Epoch 9/15000, Loss: 2061.6116491527805
Epoch 10/15000, Loss: 2050.8841531376556
Epoch 11/15000, Loss: 2042.432351546338
Epoch 12/15000, Loss: 2035.613677594839
Epoch 13/15000, Loss: 2029.991151825126
Epoch 14/15000, Loss: 2025.258933493269
Epoch 15/15000, Loss: 2021.1992980284656
Epoch 16/15000, Loss: 2017.653708528682
Epoch 17/15000, Loss: 2014.5048261048448
Epoch 18/15000, Loss: 2011.6649446469362
Epoch 19/15000, Loss: 2009.066947795632
Epoch 20/15000, Loss: 2006.6594431922554
Epoch 21/15000, Loss: 2004.4026701475043
Epoch 22/15000, Loss: 2002.2656003397267
Epoch 23/15000, Loss: 2000.222091884245
Epoch 24/15000, Loss: 1998.253053252326
Epoch 25/15000, Loss: 1996.3419794819

## Relationship between max error and epochs

In [7]:
max_errors = list(range(1000, 3000, 100))
epochs_amount: list[int] = []
for err in max_errors:    
    dl = LRNN(n, 50, 0.001)
    dl.train(train, 15000, learn_by_loss=True, max_loss=err)
    epochs_amount.append(dl.epoch)
fig = go.Figure()
fig.add_trace(go.Scatter(x=max_errors, y=epochs_amount, mode='lines+markers'))
fig.update_layout(title='Зависимость количества итераций обучения от значения максимальной ошибки',
                  xaxis_title='Значение максимальной ошибки',
                  yaxis_title='Количество итераций обучения',
                  height=720, width=1280,
                  font=dict(size=20))
fig.show()

Epoch 1/15000, Loss: 3608.5295137487674
Epoch 2/15000, Loss: 2831.393091860828
Epoch 3/15000, Loss: 2445.6501975708384
Epoch 4/15000, Loss: 2189.610198404653
Epoch 5/15000, Loss: 1998.6628733036143
Epoch 6/15000, Loss: 1848.2073201360467
Epoch 7/15000, Loss: 1725.5423209058497
Epoch 8/15000, Loss: 1622.932632454669
Epoch 9/15000, Loss: 1535.3224302796764
Epoch 10/15000, Loss: 1459.2878391907122
Epoch 11/15000, Loss: 1392.441537270086
Epoch 12/15000, Loss: 1333.0674984326404
Epoch 13/15000, Loss: 1279.8916112472994
Epoch 14/15000, Loss: 1231.9380246764856
Epoch 15/15000, Loss: 1188.438881413529
Epoch 16/15000, Loss: 1148.7746636150202
Epoch 17/15000, Loss: 1112.4373567813095
Epoch 18/15000, Loss: 1079.0040914921021
Epoch 19/15000, Loss: 1048.119227521827
Epoch 20/15000, Loss: 1019.4815013460297
Epoch 21/15000, Loss: 992.8345561082967
Epoch 1/15000, Loss: 3583.3355312164113
Epoch 2/15000, Loss: 2867.169389640843
Epoch 3/15000, Loss: 2487.345383609904
Epoch 4/15000, Loss: 2226.38346862704

In [11]:
max_errors = list(range(1000, 3000, 100))
all_errors: list[float] = []
epochs_amount_mean: list[int] = []
epochs_amount: list[int] = []
fig = go.Figure()
experiments_amount = 5
for err in max_errors:
    epoch_sum = 0
    for i in range(experiments_amount):
        all_errors.append(err)        
        dl = LRNN(n, 50, 0.001)
        dl.train(train, 15000, learn_by_loss=True, max_loss=err)
        epoch_sum += dl.epoch
        epochs_amount.append(dl.epoch)
    epochs_amount_mean.append(epoch_sum / experiments_amount)
fig.add_trace(go.Scatter(x=all_errors, y=epochs_amount, mode='markers',
                         name='Все наблюдения'))
fig.add_trace(go.Scatter(x=max_errors, y=epochs_amount_mean, mode='lines+markers',
                         name='Средние значения'))
fig.update_layout(title='Зависимость количества итераций обучения от значения максимальной ошибки',
                  xaxis_title='Значение максимальной ошибки',
                  yaxis_title='Количество итераций обучения',
                  height=720, width=1280,
                  font=dict(size=20))
fig.show()

Epoch 1/15000, Loss: 3493.7574395306265
Epoch 2/15000, Loss: 2697.4865621569
Epoch 3/15000, Loss: 2320.797913272425
Epoch 4/15000, Loss: 2074.832106966337
Epoch 5/15000, Loss: 1893.4521413920934
Epoch 6/15000, Loss: 1751.3487890587812
Epoch 7/15000, Loss: 1635.7131651595937
Epoch 8/15000, Loss: 1539.0031460460216
Epoch 9/15000, Loss: 1456.396673967814
Epoch 10/15000, Loss: 1384.6570469310827
Epoch 11/15000, Loss: 1321.5320135913532
Epoch 12/15000, Loss: 1265.4019829669153
Epoch 13/15000, Loss: 1215.065685194818
Epoch 14/15000, Loss: 1169.6069757893547
Epoch 15/15000, Loss: 1128.3098428721848
Epoch 16/15000, Loss: 1090.6031095339965
Epoch 17/15000, Loss: 1056.022874586804
Epoch 18/15000, Loss: 1024.186333218181
Epoch 19/15000, Loss: 994.7728799494018
Epoch 1/15000, Loss: 3549.5877717970125
Epoch 2/15000, Loss: 2788.4220366964087
Epoch 3/15000, Loss: 2392.6310774188473
Epoch 4/15000, Loss: 2130.6339725360276
Epoch 5/15000, Loss: 1938.3257600739353
Epoch 6/15000, Loss: 1789.0166596812646


## Relationship between image and epochs

In [ ]:
# Collecting everything

block_width = 10
block_height = 10

n = block_height * block_width
# Hidden layer neuron amount
p = 50 

img = Image.open('images/winter.jpg')
img_array = np.asarray(img)
shape = img_array.shape
blocks = image_to_blocks(img_array, block_height, block_width, overlap=0)

l = len(blocks)
# Compression coeff
print('Z =', (n*l) / ((n+l) * p+2))

color_df = ((2 * blocks / MAX_RGB_VALUE) - 1).reshape(len(blocks), -1, 3).transpose(0, 2, 1).reshape(-1, block_height * block_width)
train = np.matrix(color_df[np.random.choice(color_df.shape[0], int(color_df.shape[0] * 0.05))])

network = LRNN(n, p, 0.001)
network.train(train, 15000, learn_by_loss=True, max_loss=1500)

print(network.epoch)

# compressed = compress_image(network.W_enc, img_array, COLOR_CHANNELS_AMOUNT, block_height, block_width, 0)
# dimg = decompress_image(network.W_dec, compressed, shape, COLOR_CHANNELS_AMOUNT, block_height, block_width, 0)
# dimg_array = np.asarray(dimg)
# dimg.save('compression-decompression_test.jpg')

Z = 1.9230473377332657
Epoch 1/15000, Loss: 5209.6318296366
Epoch 2/15000, Loss: 3162.699429785375
Epoch 3/15000, Loss: 2710.4352869350473
Epoch 4/15000, Loss: 2450.4228952694975
Epoch 5/15000, Loss: 2269.9157952415144
Epoch 6/15000, Loss: 2136.711794596953
Epoch 7/15000, Loss: 2032.7831097109295
Epoch 8/15000, Loss: 1947.5842874855157
Epoch 9/15000, Loss: 1876.4486331339565
Epoch 10/15000, Loss: 1814.9423286750928
Epoch 11/15000, Loss: 1761.5728006060206
Epoch 12/15000, Loss: 1713.8186292179817
Epoch 13/15000, Loss: 1670.877801951604
Epoch 14/15000, Loss: 1631.9953954299272
Epoch 15/15000, Loss: 1596.7104192915301
Epoch 16/15000, Loss: 1564.6706844158416
Epoch 17/15000, Loss: 1534.7862598245742
Epoch 18/15000, Loss: 1506.9203029658981
Epoch 19/15000, Loss: 1481.1870920688068
19
